In [1]:
import gdspy
import numpy as np

Load the gds file and create a target file to save edited design

In [2]:
gdsii = gdspy.GdsLibrary()
file = '/Users/wendy/Desktop/Wendy-qiskit-code/export_test_0506_LL.gds'
#read the design file to meet compliance
gdsii.read_gds(file, units = 'import')

#create a new file 
gds_new = gdspy.GdsLibrary(unit = 1e-6, precision = 5e-10)

create a new cell to put the gds design

In [3]:
top = gds_new.new_cell('TOP')

Specify layer numbers

In [4]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60
chip_edge_layer = 703
text_layer1 = 98

layer_numbers  = [qubit_layer,junction_layer,ab_layer,ab_square_layer,junction_area_layer]

specify target datatype

In [5]:
data_type = 0

In [6]:
#snap to grid function that meets the layer requirements 
def snap_to_grid(coord, grid_size):
    return np.round(coord / grid_size) * grid_size

In [7]:
qubit_layer_grid = 50*1e-6
junction_layer_grid = 5*1e-6

Get the polygons from the old design

In [8]:
polys = gdsii.top_level()[0].get_polygons(by_spec = True)

In [9]:
keys = list(polys.keys())

In [10]:
keys

[(5, 10), (5, 11), (5, 0), (20, 10), (60, 10), (30, 10), (31, 10)]

Extract all the data types for a given layer number

In [11]:
datatypes = {}
for i in keys:
    try:
        print(datatypes[i[0]])
    except:
        datatypes[i[0]] = [i[1]]
    else:
        datatypes[i[0]].append(i[1])


[10]
[10, 11]


In [12]:
datatypes

{5: [10, 11, 0], 20: [10], 60: [10], 30: [10], 31: [10]}

enumerate over layer numbers and save the polygon into the new design with desired layer number

In [13]:
for i, layer in enumerate(layer_numbers):
    for j in datatypes[layer]:
        source_polyons = polys.get((layer,j),[])

        for polygon in source_polyons:
            if (layer ==5) or (layer == 31) or (layer == 30):
                polygon = snap_to_grid(polygon, qubit_layer_grid)
            elif (layer == 20) or (layer == 60):
                polygon = snap_to_grid(polygon, junction_layer_grid)
            p =gdspy.Polygon(polygon*(1e-3/1e-6), layer = int(layer), datatype = 0)
            # print(cell_names[i])
            top.add(p)

add the chip edge

In [15]:
rectangle = gdspy.Polygon(1e3*np.array([[-10.15/2,10.15/2],[10.15/2,10.15/2],[10.15/2,-10.15/2],[-10.15/2,-10.15/2]]), layer = int(chip_edge_layer))
top.add(rectangle)

Write necessary information in the information cells

In [18]:
htest1 = gdspy.Text('Email',1/2*10,(-5,4), layer = text_layer1)
htest2 = gdspy.Text('wendywan@stanford.edu',1/2*10, (-5,2), layer = text_layer1)
htest3 = gdspy.Text('Jc = 0.1uA/cm2',1/2*10,(-5,0), layer = text_layer1)

In [19]:
top.add(htest1)
top.add(htest2)
top.add(htest3)

In [20]:
gds_new.write_gds('design_final.gds')